In [9]:
!pip3 install seaborn

  Using cached seaborn-0.10.0-py3-none-any.whl (215 kB)


In [10]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (16,10)

# Data cleaning

In [47]:
data = pd.read_csv("./combined-resale-flat-prices.csv")

In [48]:
data.shape

(101741, 13)

combined-data.csv
combined-resale-flat-prices.csv
data-from-jan-2015-to-dec-2016.csv
data-jan-2017-onwards.csv
resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv
resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv
resale-flat-prices-based-on-registration-date-from-jan-2017-onwards-v2.csv


In [16]:
data = pd.read_csv("../data/combined-data.csv")

/home/tuopelto/projects/singapore-hdb-price-prediction/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
data.shape

(101741, 16)

In [18]:
data.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,mrt_station,Dist_to_station,noise,Park_dist,Bus_stop_dist
0,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60.0,Improved,1986,70,255000.0,YIO CHU KANG MRT STATION,823.905301,69.9,217.561750,207.002126
1,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1981,65,275000.0,ANG MO KIO MRT STATION,808.563653,69.9,701.207107,59.866879
2,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69.0,New Generation,1980,64,285000.0,YIO CHU KANG MRT STATION,1204.292767,69.9,239.915650,52.872599
3,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68.0,New Generation,1979,63,290000.0,BISHAN MRT STATION,1121.890344,69.9,662.396968,10.290987
4,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,64,290000.0,BISHAN MRT STATION,1121.890344,69.9,662.396968,10.290987


In [ ]:
data = data.dropna()

In [ ]:
X = data.drop(columns=['resale_price'])
y = data['resale_price']

In [ ]:
for c in X.columns:
    print(X[c].isna().sum())
    print(c, X[c].isnull().any())

In [ ]:
# Categorical features to numeric features

for c in ['month', 'town', 'flat_type', 'block', 'storey_range', 'flat_model', 'mrt_station']:
    X = pd.concat([X, pd.get_dummies(X[c])], axis=1)


tmp = X['remaining_lease'].str.split(" ", n = 1, expand = True)
X["years_left"] = tmp[0]
X["years_left"] = pd.to_numeric(X["years_left"])

X = X.drop(columns=['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model', 'remaining_lease', 'mrt_station'])

In [ ]:
X.head()

In [ ]:
# Train, validation, and test split
train_split = 0.8
validation_split = 0.1
test_split = 0.1

train_X = X.iloc[:round(len(data)*train_split),:]
train_y = y.iloc[:round(len(data)*train_split)]

validation_X = X.iloc[round(len(data)*train_split):round(len(data)*train_split + len(data)*validation_split)]
validation_y = y.iloc[round(len(data)*train_split):round(len(data)*train_split + len(data)*validation_split)]


test_X = X.iloc[round(len(data)*train_split + len(data)*validation_split):,:]
test_y = y.iloc[round(len(data)*train_split + len(data)*validation_split):]

In [ ]:
train_X.shape, train_y.shape, test_X.shape, test_y.shape

# Modeling

In [ ]:
model = RandomForestRegressor(verbose=3)
model.fit(train_X, train_y)

# Validation

In [ ]:
test_predictions = model.predict(test_X)

In [ ]:
mae = mean_absolute_error(y_true=test_y, y_pred=test_predictions)
mae, mae/np.mean(test_y)

In [ ]:
r2 = r2_score(y_true=test_y, y_pred=test_predictions)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))

plt.title(f"Correlation between target and predictions | R^2: {round(r2, ndigits=2)}", fontsize=20)
plt.xlabel("Predicted prices")
plt.ylabel("Target prices")

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

sns.regplot(test_predictions, test_y)